In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D,MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import pandas as pd

In [ ]:
#To check GPU Utilization 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Exploring the Meta data

In [ ]:
#Read the csv meta data 
df_covid_raw = pd.read_csv('D:\JupyterNotebook\CovidXRayImages\covid-chestxray-dataset\metadata.csv')

In [ ]:
#Chec count of every finding to find the number of Images 
df_covid_raw.groupby("finding").count()

In [ ]:
df_covid = df_covid_raw[['finding','filename','modality','view']]

In [ ]:
df_covid

In [ ]:
#To find the unique values in finding column
df_covid['finding'].unique()

In [ ]:
filepath = 'D:\\JupyterNotebook\\CovidXRayImages\\'

In [ ]:
#Moving X -RAY images of Covid patients to Covid Folder
for i in range(len(df_covid)) : 
  if ((df_covid.iloc[i, 0]).find('COVID-19') != -1) and ((df_covid.iloc[i, 2]).find('X-ray') != -1)and ((df_covid.iloc[i, 3]).find('L') != -1):
    print( df_covid.iloc[i, 1])
    current_path =  os.path.join(filepath,df_covid.iloc[i, 1]) 
    destination_path = os.path.join(filepath,'Covid',df_covid.iloc[i, 1]) 
    shutil.move(current_path,destination_path)
        


# Prepare Data for training

In [ ]:
# Dclaring the file paths 
covid_path = 'D:\\JupyterNotebook\\CovidXRayImages\\covid\\'
normal_path = 'D:\\JupyterNotebook\\CovidXRayImages\\normal\\'


In [ ]:
#Listing the directory
covid = os.listdir(covid_path)
normal = os.listdir(normal_path)


# Image Pre Processing

In [ ]:
#Declaring the classes
class_names = ['Normal', 'Covid']

In [ ]:
# Pre Processing the image 
data = []
label_List = []
# Image processing for Covid images
for image in covid:
    imagepath = os.path.join(covid_path,image)
    #print(imagepath)
    image = cv2.imread(imagepath)
    image = cv2.resize(image,(224,224)) # Resizing the image to 224,224
    #Check if the image is grayscale and will convert into a image with 3 channels
    if image.shape[2] ==1:
         image = np.dstack([image, image, image])
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    label = to_categorical(1, num_classes=2,dtype='int8') #Declare the Class of the image
    data.append(image) # Append the image pixels of the image 
    label_List.append(label) # Append the label of the image 
    
# Image processing for Normal images    
for image in normal:
    imagepath = os.path.join(normal_path,image)
    image = cv2.imread(imagepath)
    image = cv2.resize(image,(224,224))
    #Check if the image is grayscale and will convert into a image with 3 channels
    if image.shape[2] ==1:
         image = np.dstack([image, image, image])
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    label = to_categorical(0, num_classes=2,dtype='int8')#Declare the Class of the image
    data.append(image) # Append the image pixels of the image 
    label_List.append(label) # Append the label of the image 
        
data = np.array(data) / 255.0 # best for the model to learn if the images pixels are in the range of 0-1
label = np.array(label_List, dtype=int)


In [ ]:
# partition the data into training(70%) ,testing(10%) and validation(20%) 
(traindata, testdata, trainlabel, testlabel) = train_test_split(data, label,
	test_size=0.10, stratify=label, random_state=42)
(traindata, valdata, trainlabel, vallabel) = train_test_split(traindata, trainlabel,
	test_size=0.20, stratify=trainlabel, random_state=42)

# Data Augmentation 
Data augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data. Data augmentation techniques such as cropping, padding, and horizontal flipping are commonly used to train large neural networks.

In [ ]:
#Augmenting data by rotaing the image by 10 degree and wit fill mode 
imagegenerator = ImageDataGenerator(rotation_range = 10,fill_mode = 'nearest')

# Creating Model

In [ ]:
# load the VGG16 network, ensuring the head FC layer sets are left off
firstModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
topModel = firstModel.output
topModel = MaxPooling2D(pool_size=(4, 4))(topModel)
topModel = Flatten(name="flatten")(topModel)
topModel = Dense(64, activation="relu")(topModel)
topModel = Dropout(0.2)(topModel)
topModel = Dense(2, activation="softmax")(topModel)



In [ ]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=firstModel.input, outputs=topModel)

In [ ]:
for layer in firstModel.layers:
	layer.trainable = False

In [ ]:
model.summary()

# Optimize the Model
Optimizers are algorithms or methods used to change the attributes of your neural network such as weights and learning rate in order to reduce the losses. 

In [ ]:
LR = 1e-3
Epoch = 25
BatchSize = 5
opt = Adam(lr=LR, decay=LR / Epoch)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
History = model.fit_generator(
	imagegenerator.flow(traindata, trainlabel, batch_size=BatchSize),
	steps_per_epoch = len(traindata) // BatchSize,
	validation_data= (valdata, vallabel),
	validation_steps=len(valdata)  // BatchSize,
	epochs=Epoch)


# Training Graph 

In [ ]:
acc = History.history['accuracy']
val_acc = History.history['val_accuracy']

loss = History.history['loss']
val_loss = History.history['val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# make predictions on the testing set
predIdxs = model.predict(testdata, batch_size=BatchSize)

In [ ]:
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
model.save('CovidModel')

In [ ]:
# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testlabel.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

In [ ]:
test_loss, test_acc = model.evaluate(testdata,testlabel, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(testdata)

In [ ]:
print(predictions[0])
print(testlabel[0])

# Predicting and plotting the Images Indicating the classes

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  print(true_label[i])
  true_label = true_label[1]
  print("true label",true_label)
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  print("predicted_label",predicted_label)
  if predicted_label == 0:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)



In [ ]:
i = 1
plt.figure(figsize=(10,5))
plt.plot(1,2)
print("predictions[i]",predictions[i])
plot_image(i, predictions[i], testlabel, testdata)
plt.show()